# Sentinel Stream — Concept Drift Detector (EWMA)

This notebook adds a lightweight **concept drift** detector using EWMA (Exponential Weighted Moving Average).

- generate stream with a mean shift
- EWMA tracking + control limits
- alert points + detection delay

CPU-only; outputs saved when executed.

In [1]:
import numpy as np
import pandas as pd

SEED = 1337
rng = np.random.default_rng(SEED)
pd.set_option('display.max_columns', 50)

In [2]:
n = 5000
drift_at = 3000
x = rng.normal(0.0, 1.0, size=n)
x[drift_at:] += 1.2  # mean shift
df = pd.DataFrame({'t': np.arange(n), 'metric': x})
df.head(), drift_at

(   t    metric
 0  0  0.038268
 1  1  0.473936
 2  2 -0.137746
 3  3 -1.389334
 4  4  2.520119,
 3000)

## EWMA detector

In [3]:
lam = 0.05
L = 3.0

mu0 = df.loc[:500, 'metric'].mean()
sigma0 = df.loc[:500, 'metric'].std(ddof=0)

z = []
s = mu0
alerts = []
for i, v in enumerate(df['metric'].values):
    s = lam * v + (1-lam) * s
    z.append(s)
    # control limits for EWMA
    t = i+1
    sigma_z = sigma0 * np.sqrt((lam/(2-lam)) * (1 - (1-lam)**(2*t)))
    ucl = mu0 + L * sigma_z
    lcl = mu0 - L * sigma_z
    alerts.append(1 if (s > ucl or s < lcl) else 0)

df['ewma'] = z
df['alert'] = alerts
df[['t','metric','ewma','alert']].head(10)

,t,metric,ewma,alert
0,0,0.038268,-0.027479,0
1,1,0.473936,-0.002408,0
2,2,-0.137746,-0.009175,0
3,3,-1.389334,-0.078183,0
4,4,2.520119,0.051732,0
5,5,-1.006391,-0.001174,0
6,6,1.856816,0.091725,0
7,7,-2.502378,-0.037980,0
8,8,0.148336,-0.028664,0
9,9,-0.086123,-0.031537,0


In [4]:
first_alert = int(df.query('t >= @drift_at and alert==1')['t'].min()) if (df.query('t >= @drift_at and alert==1').shape[0] > 0) else None
delay = (first_alert - drift_at) if first_alert is not None else None
first_alert, delay

(3016, 16)